In [1]:
!pip install folium
!pip install pandas

In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
'''
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)
'''

import requests
import io

URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
response = requests.get(URL)
spacex_csv_file = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)
spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
# Initialize the map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site and add a Circle and Marker
for _, row in launch_sites_df.iterrows():
    # Extract coordinates
    coordinates = [row['Lat'], row['Long']]

    # Add Circle
    circle = folium.Circle(
        location=coordinates,
        radius=500,  # Radius in meters
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_child(folium.Popup(row['Launch Site']))

    # Add Marker with launch site name
    marker = folium.map.Marker(
        location=coordinates,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#000;"><b>{row["Launch Site"]}</b></div>'
        )
    )

    # Add Circle and Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


In [8]:
# Initialize the map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each row in the spacex_df
for _, row in spacex_df.iterrows():
    # Extract coordinates and success/failure
    coordinates = [row['Lat'], row['Long']]
    launch_status = row['class']  # 1 for success, 0 for failure

    # Define circle color based on launch status
    circle_color = 'green' if launch_status == 1 else 'red'

    # Add Circle with success/failure color
    circle = folium.Circle(
        location=coordinates,
        radius=100,  # Radius in meters
        color=circle_color,
        fill=True,
        fill_color=circle_color
    ).add_child(folium.Popup(f"{row['Launch Site']} - {'Success' if launch_status == 1 else 'Failure'}"))

    # Add Circle to the map
    site_map.add_child(circle)

# Display the map
site_map


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [10]:
marker_cluster = MarkerCluster()


In [11]:
# Create a new column 'marker_color' based on the 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Display the updated dataframe
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [12]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Iterate through each row in spacex_df
for index, record in spacex_df.iterrows():
    # Extract coordinate values (Lat, Long)
    coordinates = [record['Lat'], record['Long']]
    # Create a Marker object with its coordinate and customized icon based on success or failure
    marker = folium.Marker(
        location=coordinates,
        icon=folium.Icon(color='orange', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}"
    )
    # Add the marker to the marker_cluster
    marker_cluster.add_child(marker)

# Display the map
site_map


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [20]:

# Function to calculate distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Launch site coordinates (e.g., Kennedy Space Center Launch site)
launch_site_lat = 28.5721
launch_site_lon = -80.648

# Closest coastline coordinates (e.g., Coastline near KSC)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Initialize the map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add the launch site to the map
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue")
).add_to(site_map)

# Add coastline point to the map
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    popup=f"Coastline (Distance: {distance_coastline:.2f} km)",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} km</b></div>'
    )
)

# Add the coastline marker to the map
site_map.add_child(coastline_marker)

# Add MousePosition plugin for real-time coordinates
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = folium.plugins.MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add MousePosition to the map
site_map.add_child(mouse_position)

# Show the map
site_map


In [14]:
import folium

# Coordinates of the launch site and the coastline
launch_site_lat = 28.5721
launch_site_lon = -80.648
coastline_lat = 28.56367
coastline_lon = -80.57163

# Initialize the map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue")
).add_to(site_map)

# Add a marker for the coastline
folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="green")
).add_to(site_map)

# Create a PolyLine between the launch site and the coastline
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=1,
    color="blue"
)

# Add the PolyLine to the map
site_map.add_child(lines)

# Show the map
site_map


In [15]:
import folium
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance between two points using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Approximate radius of Earth in km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Coordinates for the launch site and closest city/railway/highway
launch_site_lat = 28.5721
launch_site_lon = -80.648
closest_point_lat = 28.610
closest_point_lon = -80.565

# Calculate the distance
distance_to_closest_point = calculate_distance(
    launch_site_lat, launch_site_lon, closest_point_lat, closest_point_lon
)

# Initialize the map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue")
).add_to(site_map)

# Add a marker for the closest point (city, railway, highway)
folium.Marker(
    [closest_point_lat, closest_point_lon],
    popup=f"Closest Point<br>{distance_to_closest_point:.2f} KM",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color: #d35400;"><b>{distance_to_closest_point:.2f} KM</b></div>',
    )
).add_to(site_map)

# Draw a PolyLine between the launch site and the closest point
line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [closest_point_lat, closest_point_lon]],
    weight=2,
    color="red"
)

# Add the PolyLine to the map
site_map.add_child(line)

# Display the map
site_map
